# Testing your Scobra knowledge

 The following tutorial will test your knowledge of Scobra. It is important to have read
 the documentation before you proceed with using this. This is not a graded exam, and you are free to consult
 the Scobra documentation when using this.
<br>
 If any errors come up, try your best to investigate the problem by consulting
 the documentation first, then searching google for the specific error. In the worst case, open the solution cell to see the answer.
 Understand the solution regardless if you got the question correct or wrong. If you have any questions about a specific topic here, feel free to
 approach your supervisor.
 <br>
 Remember, this tutorial basically follows the documentation, from loading to modifying the model. Remember to check and read the documentation. Goodluck!

## I. Basic Commands

You will need to import scobra first for all of your projects. Then, create an empty model by initializing an instance of scobra.Model

In [1]:
import scobra
m=scobra.Model()
# m is the instance of scobra.Model

Add a cell below this by clicking on this then clicking add cell on the top left of jupyter.
 Then, show the reactions present in the model. It should return a "[]" because it's an empty model. Remember to check your spelling and capitalization!

 Let's add some reactions! Recall the format of a adding reactions. (Page 11 of Documentation)
<br>
 Recall that positive coefficient means the metabolite is a product, negative coefficient means it is a reactant, and zero coefficient means the metabolite is absent in the reaction. <br>
 Create a new reaction with the name 'R1' with metabolites 'A' and 'B', whose coefficients are 1, 0, respectively.
 Next, make another one named 'R2' with the same Metabolite names but whose coefficients are -2, 1, respectively.
 Create 'R3' with the same Metabolites with coefficients -1,0, respectively.
 Finally, make 'R4' with the same Metabolites with coefficients 0,-1, respectively.
<br>
 Models have hundreds of these, and we're only doing this to build a simple model.


$$
R1\quad \rightarrow A \\
R2\quad 2A \rightarrow B\\ 
R3\quad A \rightarrow \\
R4\quad  \rightarrow B
$$

In [2]:
#Syntax to add reactions to the model
m.AddReaction('R1',{'A':1,'B':0})
m.AddReaction('R2',{'A':-2,'B':1})
m.AddReaction('R3',{'A':-1,'B':0})
m.AddReaction('R4',{'A':0,'B':1})

In [3]:
#Run these four commands after your created them to see the difference
m.PrintReaction('R1')
m.PrintReaction('R2')
m.PrintReaction('R3')
m.PrintReaction('R4')

m.GetReaction('R1')
# You can see that 'R1' doesn't have an upper bound of 1000. This is the default constraint for reactions.

R1	 --> A
R2	2 A --> B
R3	A --> 
R4	 --> B


Reaction identifier,R1
Name,
Memory address,0x07fe6b1762f40
Stoichiometry,--> A -->
GPR,
Lower bound,0.0
Upper bound,1000000


 Let's now set a constraint for 'R1'. To do so, follow 2.3.1 - 2.3.3 (pp. 18-19) in the Documentation.
<br>
 Set the lower bound to 10, and upper bound to 10.
<br>
Now set objective to 'R1' and set the direction to maximization. Here, we want to maximize the flux of reaction 'R1'

In [4]:
m.SetConstraint('R1',10,10) #(Reaction Name, Lower Bound, Upper Bound)
m.SetObjective(['R1'])
m.SetObjDirec('Max') #Another objective direction is 'Min'

 Now we're ready to solve the system. Run the solve function and GetSol to get a list of solution fluxes.

In [5]:
m.Solve()

optimal


$m.Solve()$ status is 'optimal' which means we have found the fluxes that maximize the objective function. Another possible status is 'infeasible', meaning that no combination of fluxes is in the allowable solution space.

Now let us see the optimal values of reaction fluxes:

In [6]:
m.GetSol()

{'R1': 10.0, 'R3': 10.0}

In the above dictionary, fluxes for 'R2' and 'R4' are missing, meaning they are 0. We can see their values as well by passing an extra argument in the function.

In [7]:
m.GetSol(IncZeroes=True)

{'R1': 10.0, 'R2': 0.0, 'R3': 10.0, 'R4': 0.0}

 For the curious, this is represented by a Metabolites x Reactions matrix multiplied by a vector given by the objective function. Multiplying this matrix to this vector will give a single number (0).
 <br>
 We are finding out the values of x that satisfy this requirement.

$$
\begin{matrix}
A\\   
B\\
\end{matrix}
\begin{matrix}
R1\quad R2\quad R3\quad R4\\
\begin{pmatrix}
1 & -2 & -1 & 0\\
0 & 1 & 0 & -1\\
\end{pmatrix}
\end{matrix}
\times
\begin{pmatrix}
x_1\\
x_2\\
x_3\\
x_4\\
\end{pmatrix}
=0
$$

 After solving it given the flux constraints,
 <br>
 $x_1 = 10$
 <br>
 $x_2= 0$
 <br>
 $x_3=10$
 <br>
 $x_4 =0$

## II. Using a Toy Example

Let us now apply our commands on the CAM model to generate biomasses. This section shows the general steps you would do for working on most plants in this research group. Due to the code complexity, code will be posted here along with instructions to modify the code. You must modify the code correctly to make it run.

### Step 1. Import libraries and define project directory
You should have the CAM model in the directory containing this tutorial. Since it's in the same directory, you only need to put the filename into the EDIT below. Make sure to include the filetype (.xls) at the end!

In [ ]:
import scobra
import os
#For DataFrames and saving them as .csv (Will be explained later)
import pandas as pd

#Manually put in the filename that you want to use in between the '' below
m=scobra.Model('EDIT') #Edit here (a filename with .xls)

#Creates a Project directory. Feel free to change the name
sim = 'CAM'
if not os.path.isdir(sim):
    os.mkdir(sim)
sim = sim + '/' + sim

### Step 2. Define constraints

We first set the constraints that we use for many models. The constraints_list is not actually a list, but it uses Python's dictionary data type in the format {Constraint:Parameters}. Calling constraints_list['Constraint'] will output the corresponding parameters. However, calling constraints_list[(0,0)] will output an error because there are multiple corresponding contraints. This is a general feature of the dictionary. More info on the dictionary datatype here: https://realpython.com/python-dicts/

In [ ]:
### Setting general model constraints
constraints_list = {
                   'Starch_biomass1':(0,0),
                   'Starch_biomass2':(0,0),
                   'NADPH_Dehydrogenase_p1':(0,0),
                   'NADPH_Dehydrogenase_p2':(0,0),
                   'Plastoquinol_Oxidase_p1':(0,0),
                   'Plastoquinol_Oxidase_p2':(0,0),
                   'SUCROSE_v_dielTransfer':(0,0),
                   'FRUCTAN_v_dielTransfer':(0,0),
                   }
#SetConstraints accepts a dictionary with the appropriate values
m.SetConstraints(constraints_list)

#Or you can also individually set each constraint
m.SetConstraints({'Photon_tx2':(0,0)})
#Set the Lower Bound and Upper Bound as 8.5
m.SetConstraints({'ATPase_tx1':(8.5/9,8.5/9)})
m.SetConstraints({'ATPase_tx2':(8.5/9,8.5/9)})
m.SetConstraints({'NADPHoxc_tx1':(8.5/9,8.5/9)})
m.SetConstraints({'NADPHoxc_tx2':(8.5/9,8.5/9)})
m.SetConstraints({'NADPHoxm_tx1':(8.5/9,8.5/9)})
m.SetConstraints({'NADPHoxm_tx2':(8.5/9,8.5/9)})
m.SetConstraints({'NADPHoxp_tx1':(8.5/9,8.5/9)})
m.SetConstraints({'NADPHoxp_tx2':(8.5/9,8.5/9)})
m.SetConstraints({'GLC_tx1':(0,0)})
m.SetConstraints({'GLC_tx2':(0,0)})
m.SetConstraints({'SO4_tx1':(0,None),'SO4_tx2':(0,None),'Pi_tx1':(0,None),'Pi_tx2':(0,None)})
m.SetConstraints({'Biomass_tx1':(0,0),'Biomass_tx2':(0,0)})
m.SetConstraints({'Sucrose_tx1':(0,0),'Sucrose_tx2':(0,0)})
m.SetConstraints({'H_tx1':(None,None),'H_tx2':(None,None)})
m.SetConstraints({'NH4_tx1':(0,0),'NH4_tx2':(0,0)})
m.SetConstraints({'unlProtHYPO_c1':(0,0),'unlProtHYPO_c2':(0,0)})
m.SetConstraints({'H_ic1':(None,None),'H_ic2':(None,None)})

In [ ]:
### Setting C3 specific constraints
m.SetConstraints({'diel_biomass':(0.258565369)})
m.SetConstraints({'CO2_tx1':(None,None)})
m.SetConstraints({'O2_tx1':(None,None)})
m.SetConstraints({'CO2_tx2':(None,None)})
m.SetConstraints({'O2_tx2':(None,None)})
m.SetReacsFixedRatio({"RXN_961_p1":1, "RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p1":3})
m.SetReacsFixedRatio({"RXN_961_p2":1, "RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p2":3})

### Step 3. Set objectives and run the corresponding simulations

After defining our constraints, we are ready to set our objective and run the simulation to check if it produces any solutions. We then save the results to csv files with the appropriate filenames

In [ ]:
#We want to minimize the amount of flux produced
m.SetObjDirec('EDIT') #Edit here ('Min' for minimize and 'Max' for maximize)
m.SetObjective({'Photon_tx1':1,'Photon_tx2':1})

#Perform Flux Balance Analysis
m.MinFluxSolve()
sol=m.GetSol(AsMtx=True)
sol.ToFile(sim + '.csv')

#Perform Flux Variability Analysis
fva = m.FVA(AsMtx=True)
fva.to_csv(sim + '_FVA.csv')

We should be able to view the files inside the 'CAM' folder.

In [ ]:
#Minimization is also our objective for the rest of these simulations, so we don't have to set the objective again

### ME only simulation
m.SetConstraints({'PEPCARBOXYKIN_RXN_c1':(0,0),'PEPCARBOXYKIN_RXN_c2':(0,0)})
m.MinFluxSolve()
sol=m.GetSol(AsMtx=True)
sol.ToFile(sim + '_ME.csv')
fva = m.FVA(AsMtx=True)
fva.to_csv(sim + '_ME_FVA.csv')
m.SetConstraints({'PEPCARBOXYKIN_RXN_c1':(0,None),'PEPCARBOXYKIN_RXN_c2':(0,None)})

### PEPCK only simulation
m.SetConstraints({'MALIC_NADP_RXN_c1':(0,0),'MALIC_NADP_RXN_c2':(0,0),'MALIC_NADP_RXN_p1':(0,0),'MALIC_NADP_RXN_p2':(0,0),'MALIC_NAD_RXN_m1':(0,0),'MALIC_NAD_RXN_m2':(0,0)})
m.MinFluxSolve()
sol=m.GetSol(AsMtx=True)
sol.ToFile(sim + '_PEPCK.csv')
fva = m.FVA(AsMtx=True)
fva.to_csv(sim + '_PEPCK_FVA.csv')
m.SetConstraints({'MALIC_NADP_RXN_c1':(0,None),'MALIC_NADP_RXN_c2':(0,None),'MALIC_NADP_RXN_p1':(0,None),'MALIC_NADP_RXN_p2':(0,None),'MALIC_NAD_RXN_m1':(0,None),'MALIC_NAD_RXN_m2':(0,None)})

### free carbohydrate storage simulation
m.SetConstraints({'SUCROSE_v_dielTransfer':(None,None)})
m.SetConstraints({'FRUCTAN_v_dielTransfer':(None,None)})
m.MinFluxSolve()
sol=m.GetSol(AsMtx=True)
sol.ToFile(sim + '_free_storage.csv')
fva = m.FVA(AsMtx=True)
fva.to_csv(sim + '_free_storage_FVA.csv')

### sucrose only simulation
m.SetConstraints({'STARCH_p_dielTransfer':(0,0)})
m.SetConstraints({'FRUCTAN_v_dielTransfer':(0,0)})
m.MinFluxSolve()
sol=m.GetSol(AsMtx=True)
sol.ToFile(sim + '_sucrose.csv')
fva = m.FVA(AsMtx=True)
fva.to_csv(sim + '_sucrose_FVA.csv')

### fructan only simulation
m.SetConstraints({'SUCROSE_v_dielTransfer':(0,0)})
m.SetConstraints({'FRUCTAN_v_dielTransfer':(None,None)})
m.MinFluxSolve()
sol=m.GetSol(AsMtx=True)
sol.ToFile(sim + '_fructan.csv')
fva = m.FVA(AsMtx=True)
fva.to_csv(sim + '_fructan_FVA.csv')

### Step 4. Save experimental results in .csv files 

While the previous code has produced some .csv files, this step produces one summary .csv file that includes the essential information from each simulation. We use Pandas for creating the datatype that stores them (Pandas.DataFrame), which outputs .csv files. Comments will explain the basic functionality of Pandas, but it is very useful to learn other commands for any type of tabular data work. Read more Pandas here: https://www.learnpython.org/en/Pandas_Basics

In [ ]:
### Generate summary files (Supplementary Table S2)

#We imported pandas as pd so every pandas command becomes pd.[command]

#read_csv reads the .csv file and stores it as a DataFrame
ir = pd.read_csv("Interested_Rxn.csv")

starch = pd.read_csv(sim + ".csv")
starch = starch.rename(columns={starch.columns[0]:"Reactions"})

#We combine the two DataFrames by matching them with the 'Reactions' Column
ir = pd.merge(ir, starch, on=["Reactions"], how="left")
ir = ir.rename(columns={ir.columns[-1]:"Starch"})

#We do the same for every .csv produced by our simulations

pepck = pd.read_csv(sim + "_PEPCK.csv")
pepck = pepck.rename(columns={pepck.columns[0]:"Reactions"})
ir = pd.merge(ir, pepck, on=["Reactions"], how="left")
ir = ir.rename(columns={ir.columns[-1]:"PEPCK"})

me = pd.read_csv(sim + "_ME.csv")
me = me.rename(columns={me.columns[0]:"Reactions"})
ir = pd.merge(ir, me, on=["Reactions"], how="left")
ir = ir.rename(columns={ir.columns[-1]:"ME"})

free = pd.read_csv(sim + "_free_storage.csv")
free = free.rename(columns={free.columns[0]:"Reactions"})
ir = pd.merge(ir, free, on=["Reactions"], how="left")
ir = ir.rename(columns={ir.columns[-1]:"Free"})

sucrose = pd.read_csv(sim + "_sucrose.csv")
sucrose = sucrose.rename(columns={sucrose.columns[0]:"Reactions"})
ir = pd.merge(ir, sucrose, on=["Reactions"], how="left")
ir = ir.rename(columns={ir.columns[-1]:"Sucrose"})

fructan = pd.read_csv(sim + "_fructan.csv")
fructan = fructan.rename(columns={fructan.columns[0]:"Reactions"})
ir = pd.merge(ir, fructan, on=["Reactions"], how="left")
ir = ir.rename(columns={ir.columns[-1]:"Fructan"})

#We fill all empty cells with 0
#Finally, we save our final DataFrame to the summary .csv
ir = ir.fillna(0)
ir.to_csv(sim + "_Summary.csv",index=False)

'''
Note: This could be abstracted into a for loop because 
every file undergoes the same set of operations.
For the CS person inside you, give it a try.
'''

### Step 4 (Optional). Getting rid of rows with all zeros and saving the rest in a csv file.
The csv file we just produced contains the essential information from each simulation. However, it also contains some rows with all entries equal to '0'. To obtain a csv without these arguably insignificant rows, we can filter the data frame before converting it into a csv.


In [ ]:
ir_without_zeros = ir

for row in range(len(ir.index)):
    flag = False
    for column in range(1,len(ir.columns)):
        if ir.iat[row, column] != 0:
            flag = True
            # the moment we realize that there is a non-zero value in one of the columns, we can go ahead to the next row
            break
    if flag == False:
        # drop these rows from the data frame since they do not have a single non-zero value across the columns
        ir_without_zeros =  ir_without_zeros.drop(row)

# let's save this into a .csv file
# index=False is used to fix the indices accordingly after we removed certain rows
ir_without_zeros.to_csv(sim + 'test-summary-without-zeros-rows.csv', index = False)